In [69]:
import requests as rq
from bs4 import BeautifulSoup
#import io
import pymongo as pm
#import pandas as pd
#import urllib
import time

item_category = []
numb = 0

region = 3 #地區參數, 1=台北市, 3=新北市
page = 1
listlink = 'https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=' + str(region) +'&firstRow='
listlink = listlink.rstrip()

#下面是當要GET rsList時，需夾帶於header中的device id、cookie與CSRF-token等資料，才能順利取得data.
csrf_token = 'YN8YoYS0DTEAKQHAGTsbLnMt7nOuUZE5dOGZtV4W'  #為了對應網站的CSFR令牌機制，從chrome開發者工具中取得網站在傳送GET-rsList時使用的token.
deviceid = '7ikinjg0thbdur20s2cueh28i3'
cookie = 'webp=1; PHPSESSID=7ikinjg0thbdur20s2cueh28i3; T591_TOKEN=7ikinjg0thbdur20s2cueh28i3; regionCookieId=1; newUI=1; _ga=GA1.3.1990700666.1639128390; _ga=GA1.4.1990700666.1639128390; tw591__privacy_agree=0; new_rent_list_kind_test=0; user_index_role=1; _gid=GA1.3.2101413961.1639291466; _gid=GA1.4.2101413961.1639291466; user_browse_recent=a%3A5%3A%7Bi%3A0%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bi%3A11713382%3B%7Di%3A1%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bi%3A11738745%3B%7Di%3A2%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bi%3A11717232%3B%7Di%3A3%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bi%3A8506657%3B%7Di%3A4%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A1%3Bs%3A7%3A%22post_id%22%3Bi%3A11727231%3B%7D%7D; urlJumpIp=3; urlJumpIpByTxt=%E6%96%B0%E5%8C%97%E5%B8%82; _gat_UA-97423186-1=1; _gat=1; _dc_gtm_UA-97423186-1=1; XSRF-TOKEN=eyJpdiI6ImJvd1dGVXl2VWdPS2w0bm5wTWFabmc9PSIsInZhbHVlIjoiR1AwXC83RWlpXC8zenVWZEVNM01ORGpBMTdiWVBCbDJRd3F3Z3VWa1hUNjJQcjlXbm1PSGY3K091WFlObk1HUEFaTXVHWmh4RysxK3VNUU9sZm02NEhxUT09IiwibWFjIjoiNDMzMmU0Nzc3ODRlMGIxMDkxODk0MDRkNzUzMmFkN2M1NDk5MjcyYzk5N2VmMDM3ZWVmZGNmYzM2MDQ5ZWQ5MCJ9; 591_new_session=eyJpdiI6IjRmOEp5clVuaVliK2ZGWTRxbnBmUXc9PSIsInZhbHVlIjoiTmxwVWJFQ1lma2xzRFBrZnhPbmc3ZUxsREZOWVYyMktZZ1UzazBzcm5zb05hREdVdVNDc2VTMUlFU1lxOFwvQTFjMDZocTVPdis0Um1PSVhkdFlSSlwvUT09IiwibWFjIjoiYmJjOTE0NDZhYjQxMjVmMGI3ZjlkOWYzMTQyMDk5OTAzMjk2N2YyNGQ3ZWJlZTdiNWVmM2MxM2ZjMWZhNGE5ZSJ9'


#設置睡眠用的timer
def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;

tStart = time.time()#啟動計時器

#Python的for-loop是依循一個sequence作為其範圍，所以沒辦法像Java或C#一樣直接給定int，必須以類似 range(s,e) 的範圍產生語法產生一sequence。
for page in range(1,323):
    #先透過requests套件get所要的內容，並包含以下設定的headers內容
    print(listlink + str((page - 1)*30))
    response = rq.get(listlink + str((page - 1)*30),
                      headers={'User-Agent': 'Custom',
                               'X-CSRF-TOKEN': csrf_token,
                               'device': 'pc',
                               'deviceid': deviceid,
                               'Cookie': cookie}) #headers部分靈感來自https://stackoverflow.com/questions/47506092/python-requests-get-always-get-404
    soup = BeautifulSoup(response.text) 
    rental_items = str(soup.find('p'))   #過濾出<p>標籤內的文字並轉為string
    rental_items = rental_items.replace('<p>','')   #濾除'<p>'
    rental_items = rental_items.replace('</p>','')   #濾除'<p>'
    rental_items = rental_items.replace('<span>','')   #濾除'<p>'
    rental_items = rental_items.replace('</span>','')  #濾除妨礙轉換list的字符
    rental_items = rental_items.replace('</a>','')   #濾除'<p>'
    
    rental_items = eval(rental_items)
    
    d = rental_items['data']['data'] #利用 eval(strname) 將string轉為list
    
    for i in d:            #印出當前頁面的所有物件ID
        numb = numb + 1
        
        print(str(numb) + '. ' + str(i['post_id']) + ' :OK')
        #print(str(i['post_id']))
        item_category.append(i)
        
    print('Page '+ str(page) + 'saved.')
    page = page + 1
    time.sleep(sleeptime(0,0,3))

#print(item_category)

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=0
1. 11815877 :OK
2. 11661657 :OK
3. 11760896 :OK
4. 11671368 :OK
5. 11844095 :OK
6. 11601221 :OK
7. 11762497 :OK
8. 11719313 :OK
9. 11833350 :OK
10. 11733425 :OK
11. 11342735 :OK
12. 11743526 :OK
13. 11785173 :OK
14. 11841391 :OK
15. 11844730 :OK
16. 11751177 :OK
17. 11775494 :OK
18. 11816117 :OK
19. 11710680 :OK
20. 11848676 :OK
21. 11757520 :OK
22. 11800417 :OK
23. 11838852 :OK
24. 11778228 :OK
25. 11848131 :OK
26. 11743502 :OK
27. 11776515 :OK
28. 11818223 :OK
29. 11732608 :OK
30. 11848273 :OK
Page 1saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=30
31. 11727497 :OK
32. 11757611 :OK
33. 11748861 :OK
34. 11830703 :OK
35. 11831267 :OK
36. 11801713 :OK
37. 11800277 :OK
38. 11818136 :OK
39. 11765659 :OK
40. 11744138 :OK
41. 11835738 :OK
42. 11755699 :OK
43. 11826614 :OK
44. 11838100 :OK
45. 11753266 :OK
46. 11789275 :OK
47. 1180991

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=390
391. 11849446 :OK
392. 11711505 :OK
393. 11841498 :OK
394. 11746300 :OK
395. 11834553 :OK
396. 11831741 :OK
397. 11848468 :OK
398. 11838851 :OK
399. 11766939 :OK
400. 11803914 :OK
401. 11793073 :OK
402. 11815412 :OK
403. 11783606 :OK
404. 11833376 :OK
405. 11826317 :OK
406. 11709249 :OK
407. 11778564 :OK
408. 11766765 :OK
409. 11821169 :OK
410. 11805042 :OK
411. 11831216 :OK
412. 11741949 :OK
413. 11774094 :OK
414. 11745872 :OK
415. 11753531 :OK
416. 11834074 :OK
417. 11779410 :OK
418. 11835780 :OK
419. 11755973 :OK
420. 11742111 :OK
Page 14saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=420
421. 11720559 :OK
422. 11739917 :OK
423. 11736944 :OK
424. 11789015 :OK
425. 11841916 :OK
426. 11828681 :OK
427. 11847335 :OK
428. 11815417 :OK
429. 11768892 :OK
430. 11752775 :OK
431. 11760226 :OK
432. 11745089 :OK
433. 11749038 :OK
434. 1

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=780
781. 11740251 :OK
782. 11732284 :OK
783. 11783235 :OK
784. 11758356 :OK
785. 11790617 :OK
786. 11774831 :OK
787. 11807040 :OK
788. 11801994 :OK
789. 11806139 :OK
790. 11805397 :OK
791. 11768058 :OK
792. 11804983 :OK
793. 11726084 :OK
794. 11770645 :OK
795. 11795510 :OK
796. 11786556 :OK
797. 11798049 :OK
798. 11774550 :OK
799. 11825303 :OK
800. 11812329 :OK
801. 11805908 :OK
802. 11817700 :OK
803. 11798557 :OK
804. 11761518 :OK
805. 11783415 :OK
806. 11756196 :OK
807. 11817825 :OK
808. 11795425 :OK
809. 11794892 :OK
810. 11817892 :OK
Page 27saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=810
811. 11817892 :OK
812. 11762215 :OK
813. 11719471 :OK
814. 11823854 :OK
815. 11758356 :OK
816. 11810098 :OK
817. 11753877 :OK
818. 11796129 :OK
819. 11798158 :OK
820. 11718206 :OK
821. 11771992 :OK
822. 11803020 :OK
823. 11766964 :OK
824. 1

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=1170
1171. 11851104 :OK
1172. 11844758 :OK
1173. 11841526 :OK
1174. 11761382 :OK
1175. 11831863 :OK
1176. 11771751 :OK
1177. 11851519 :OK
1178. 11821088 :OK
1179. 11851711 :OK
1180. 11846549 :OK
1181. 11799356 :OK
1182. 11848183 :OK
1183. 11844296 :OK
1184. 11832180 :OK
1185. 11827805 :OK
1186. 11844340 :OK
1187. 11828031 :OK
1188. 11720144 :OK
1189. 11722560 :OK
1190. 11765544 :OK
1191. 11815256 :OK
1192. 11733047 :OK
1193. 11831703 :OK
1194. 11753771 :OK
1195. 11792343 :OK
1196. 11832004 :OK
1197. 11844590 :OK
1198. 11841230 :OK
1199. 11828412 :OK
1200. 11844869 :OK
Page 40saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=1200
1201. 11827805 :OK
1202. 11750514 :OK
1203. 11837403 :OK
1204. 11828031 :OK
1205. 11845091 :OK
1206. 11789085 :OK
1207. 11750643 :OK
1208. 11751144 :OK
1209. 11821059 :OK
1210. 11836664 :OK
1211. 11764241 :OK

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=1530
1531. 11844272 :OK
1532. 11829688 :OK
1533. 11802980 :OK
1534. 11797422 :OK
1535. 11796708 :OK
1536. 11776852 :OK
1537. 11848911 :OK
1538. 11792073 :OK
1539. 11810079 :OK
1540. 11732579 :OK
1541. 11786231 :OK
1542. 11809948 :OK
1543. 11777220 :OK
1544. 11766431 :OK
1545. 11827119 :OK
1546. 11805552 :OK
1547. 11812620 :OK
1548. 11737643 :OK
1549. 11797855 :OK
1550. 11707174 :OK
1551. 11752344 :OK
1552. 11806634 :OK
1553. 11739671 :OK
1554. 11819129 :OK
1555. 11657063 :OK
1556. 11847371 :OK
1557. 11743338 :OK
1558. 11755206 :OK
1559. 11816247 :OK
1560. 11743535 :OK
Page 52saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=1560
1561. 11744095 :OK
1562. 11766216 :OK
1563. 11760386 :OK
1564. 11765397 :OK
1565. 11848661 :OK
1566. 11707992 :OK
1567. 11839181 :OK
1568. 11773272 :OK
1569. 11721933 :OK
1570. 11783296 :OK
1571. 11744865 :OK

...

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8010
8011. 11736614 :OK
8012. 11736629 :OK
8013. 11834244 :OK
8014. 11743111 :OK
8015. 11762218 :OK
8016. 11715350 :OK
8017. 11834132 :OK
8018. 11723958 :OK
8019. 11733447 :OK
8020. 11732739 :OK
8021. 11831065 :OK
8022. 11757510 :OK
8023. 11759034 :OK
8024. 11757236 :OK
8025. 11758644 :OK
8026. 11762038 :OK
8027. 11757263 :OK
8028. 11760762 :OK
8029. 11762017 :OK
8030. 11753816 :OK
8031. 11758532 :OK
8032. 11759818 :OK
8033. 11761292 :OK
8034. 11759024 :OK
8035. 11761402 :OK
8036. 11762002 :OK
8037. 11757784 :OK
8038. 11518966 :OK
8039. 11759571 :OK
8040. 11757957 :OK
Page 268saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8040
8041. 11761501 :OK
8042. 11757957 :OK
8043. 11759208 :OK
8044. 11758227 :OK
8045. 11761757 :OK
8046. 10913914 :OK
8047. 11757387 :OK
8048. 11759776 :OK
8049. 11759818 :OK
8050. 11762076 :OK
8051. 11761907 :O

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8370
8371. 11824198 :OK
8372. 11800687 :OK
8373. 11824189 :OK
8374. 11824183 :OK
8375. 11824169 :OK
8376. 11812543 :OK
8377. 11803063 :OK
8378. 11788339 :OK
8379. 11824094 :OK
8380. 11767393 :OK
8381. 11824040 :OK
8382. 11824000 :OK
8383. 11823904 :OK
8384. 11823813 :OK
8385. 11823794 :OK
8386. 11701661 :OK
8387. 11819518 :OK
8388. 11823659 :OK
8389. 11823651 :OK
8390. 11823655 :OK
8391. 11810699 :OK
8392. 11778365 :OK
8393. 11823443 :OK
8394. 11744967 :OK
8395. 11823436 :OK
8396. 11823415 :OK
8397. 11752689 :OK
8398. 11752718 :OK
8399. 11728498 :OK
8400. 11823312 :OK
Page 280saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8400
8401. 11823260 :OK
8402. 11718940 :OK
8403. 11779583 :OK
8404. 11719003 :OK
8405. 11823111 :OK
8406. 11823069 :OK
8407. 11823061 :OK
8408. 11755619 :OK
8409. 11778748 :OK
8410. 11787255 :OK
8411. 11796413 :O

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8730
8731. 11740513 :OK
8732. 11739320 :OK
8733. 11742072 :OK
8734. 11741044 :OK
8735. 11738823 :OK
8736. 11741816 :OK
8737. 11739989 :OK
8738. 11741939 :OK
8739. 11742110 :OK
8740. 11741760 :OK
8741. 11741451 :OK
8742. 11738821 :OK
8743. 11742089 :OK
8744. 11738854 :OK
8745. 11739235 :OK
8746. 11739059 :OK
8747. 11711361 :OK
8748. 11711216 :OK
8749. 11711259 :OK
8750. 11797700 :OK
8751. 11807527 :OK
8752. 11784148 :OK
8753. 11765371 :OK
8754. 11776239 :OK
8755. 11800354 :OK
8756. 11681751 :OK
8757. 11781790 :OK
8758. 11729925 :OK
8759. 11811910 :OK
8760. 11811839 :OK
Page 292saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=8760
8761. 11811836 :OK
8762. 11811832 :OK
8763. 11707717 :OK
8764. 11811643 :OK
8765. 11811487 :OK
8766. 11712773 :OK
8767. 11811454 :OK
8768. 11811417 :OK
8769. 11811379 :OK
8770. 11811372 :OK
8771. 11811155 :O

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=9090
9091. 11788941 :OK
9092. 11588661 :OK
9093. 11793199 :OK
9094. 11793044 :OK
9095. 11767261 :OK
9096. 11706311 :OK
9097. 11792637 :OK
9098. 11767256 :OK
9099. 11792593 :OK
9100. 11792510 :OK
9101. 11792473 :OK
9102. 11792375 :OK
9103. 11792245 :OK
9104. 11792239 :OK
9105. 11792233 :OK
9106. 11792134 :OK
9107. 11792129 :OK
9108. 11792095 :OK
9109. 11735157 :OK
9110. 11718862 :OK
9111. 11791846 :OK
9112. 11757766 :OK
9113. 11787960 :OK
9114. 11713512 :OK
9115. 11791648 :OK
9116. 11791617 :OK
9117. 11785298 :OK
9118. 11791406 :OK
9119. 11763217 :OK
9120. 11788578 :OK
Page 304saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=9120
9121. 11791108 :OK
9122. 11791075 :OK
9123. 11791064 :OK
9124. 11706292 :OK
9125. 11752194 :OK
9126. 11790273 :OK
9127. 11712661 :OK
9128. 11786328 :OK
9129. 11789979 :OK
9130. 11712739 :OK
9131. 11732934 :O

https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=9450
9451. 11752520 :OK
9452. 11743343 :OK
9453. 11763772 :OK
9454. 11763745 :OK
9455. 11763742 :OK
9456. 11729670 :OK
9457. 11763275 :OK
9458. 11735449 :OK
9459. 11763210 :OK
9460. 11763009 :OK
9461. 11762896 :OK
9462. 11760004 :OK
9463. 11761123 :OK
9464. 11744608 :OK
9465. 11761704 :OK
9466. 11761701 :OK
9467. 11742475 :OK
9468. 11743244 :OK
9469. 11737847 :OK
9470. 11760210 :OK
9471. 11760041 :OK
9472. 11712550 :OK
9473. 11759156 :OK
9474. 11759016 :OK
9475. 11719441 :OK
9476. 11742692 :OK
9477. 11758278 :OK
9478. 11755437 :OK
9479. 11757970 :OK
9480. 11757599 :OK
Page 316saved.
https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3&firstRow=9480
9481. 11718577 :OK
9482. 11718585 :OK
9483. 11718592 :OK
9484. 11756544 :OK
9485. 11736879 :OK
9486. 11736885 :OK
9487. 11741567 :OK
9488. 11741568 :OK
9489. 11716875 :OK
9490. 11755814 :OK
9491. 11726069 :O

In [70]:
#將網頁擷取的部分模組化之測試

from pymongo import MongoClient
import time

#MongoDB
client = MongoClient('localhost', 27017)
rentaldb = client.rental_db   #主機.資料庫
newtaipeicity = rentaldb.newtaipeicity   #資料庫.colletcion



def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;

tStart = time.time()#計時開始

rqheader = {'User-Agent': 'Custom',
            'X-CSRF-TOKEN': csrf_token,
            'device': 'pc',
            'deviceid': '7ikinjg0thbdur20s2cueh28i3'}

def retrivedata(idtstring):

    #以下是html語法錯亂之校正#
    #先print到此為止的結果，並將文本與Postman解析出的正確版文本做比較，以調整錯誤的HTML標籤並符合JSON文本規範。
    idtstring = idtstring.replace('\"\\&quot;', '\\"')
    idtstring = idtstring.replace('\\&quot;', ';')
    idtstring = idtstring.replace(';;\"', '\\\"')
    ###文本調整結束###

    #print(item_data)
    idtstring = eval(idtstring)['data']
    #print(item_data)

    item_title = str(idtstring['title'])  #物件標題
    item_postid = str()
    item_contactor = str(idtstring['linkInfo']['name'])  #聯絡人(屋主： 魏先生)
    item_crole = str(idtstring['linkInfo']['roleName'])  #聯絡人身份（屋主、仲介）
    item_contactor = str(item_contactor.replace(item_crole + ': ', ''))  #聯絡人去除身份與冒號
    item_phone = str(idtstring['linkInfo']['mobile'])  #聯絡人手機
    item_shape = str(idtstring['info'][0]['value'])    #建物現況（整層住家、獨立套房）
    item_status = str(idtstring['info'][3]['value'])  #建物型態（電梯大樓）
    item_rule = str(idtstring['service']['rule'])   #物件使用規則（內含限男限女資訊）
    
    #從item_rule文本判斷限男限女，或無限制
    if ('限女' in item_rule):
        item_genderlim = '限女性'
    elif ('限男' in item_rule):
        item_genderlim = '限男性'
    else:
        item_genderlim = '無特別限制'

    return {"物件標題": item_title,
           "出租者": item_contactor,
           "出租者身份": item_crole,
           "聯絡電話": item_phone,
           "物件型態": item_status,
           "物件現況": item_shape,
           "性別要求": item_genderlim}
    

for i in item_category:
    number = 1
    itemid = i['post_id']
    itemkind = i['kind_name']
    itemurl = 'https://bff.591.com.tw/v1/house/rent/detail?id=' + str(itemid)
    #print(itemurl)
    response = rq.get(itemurl, headers = rqheader)
    soup = BeautifulSoup(response.text)

    #print(retrivedata(item_data))
    numb = numb + 1
    try:
        item_data = str(soup)
        
        #先移除影響資料的html標籤
        item_data = item_data.replace('<html>','')
        item_data = item_data.replace('<body>','')
        item_data = item_data.replace('<p>','')
        item_data = item_data.replace('</html>','')
        item_data = item_data.replace('</body>','')
        item_data = item_data.replace('</p>','')
        item_data = item_data.replace('</a>','')
        item_data = item_data.strip()

        
        
        #寫入DB ⚠️留意寫入的collection是否為正確城市
        newtaipeicity.insert_one(retrivedata(item_data))
        
        #印出 編號+物件ID+'寫入完成.'+寫入的資料
        print(str(numb) + '. ' + str(itemid) + '：Insert done.\t' + str(retrivedata(item_data)))

        #每爬完一筆物件睡1秒鐘防止被認為是ddos.
        time.sleep(sleeptime(0,0,1))

    except:
        
        print(str(numb) + '. ' + '物件' + str(itemid) + '擷取錯誤，跳過')


9644. 11815877：Insert done.	{'物件標題': '乾淨明亮全新整理套房', '出租者': '黃先生', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1545486', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9645. 11661657：Insert done.	{'物件標題': '近藍線板橋捷運-全新套房出租', '出租者': '王小姐', '出租者身份': '屋主', '聯絡電話': '0912-660-014', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9646. 11760896：Insert done.	{'物件標題': '中和遠東全新裝潢大套房，通風採光機能超優', '出租者': '林先生', '出租者身份': '屋主', '聯絡電話': '0963-669-923', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9647. 11671368：Insert done.	{'物件標題': '南勢角捷運套房中華100mWIFI第四台', '出租者': '劉小姐', '出租者身份': '屋主', '聯絡電話': '0939-377-355', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9648. 11844095：Insert done.	{'物件標題': '海山站精美裝潢兩房一廳大陽台', '出租者': '徐小姐', '出租者身份': '代理人', '聯絡電話': '0938228801', '物件型態': '公寓', '物件現況': '2房1廳1衛', '性別要求': '無特別限制'}
9649. 11601221：Insert done.	{'物件標題': '七張捷運站到站就到家（全棟禁菸）', '出租者': '屋主', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1531105', '物件型態': '電梯大樓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
9650. 11762497：Insert done.	{'物件標題': '重陽

9696. 7922591：Insert done.	{'物件標題': '元氣大鎮3房設備齊全-附傢俱', '出租者': '廖太太', '出租者身份': '屋主', '聯絡電話': '0983656152', '物件型態': '電梯大樓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
9697. 11797840：Insert done.	{'物件標題': '@全新完工一房一廳、豪華大套房', '出租者': '蔡小姐', '出租者身份': '屋主', '聯絡電話': '0918-537-688', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9698. 11705801：Insert done.	{'物件標題': '華中橋中和捷運北歐簡約風1房1廳1衛', '出租者': '孫先生', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1538477', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9699. 11713114：Insert done.	{'物件標題': '新泰路二樓公寓出租(近新莊佳瑪)', '出租者': '張女士', '出租者身份': '屋主', '聯絡電話': '0935-546-899', '物件型態': '公寓', '物件現況': '3房2廳1衛', '性別要求': '無特別限制'}
9700. 11718246：Insert done.	{'物件標題': '近台北市捷運電梯套房-1個月起租免押金-', '出租者': '陳先生', '出租者身份': '屋主', '聯絡電話': '0963-655-588', '物件型態': '電梯大樓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9701. 11782557：Insert done.	{'物件標題': '共享空間超大/機會難得/僅剩1間雅房', '出租者': '鄭小姐', '出租者身份': '代理人', '聯絡電話': '0952-809-046', '物件型態': '公寓', '物件現況': '雅房', '性別要求': '限女性'}
9702. 11781796：Insert done.	{'物件標題': '新店建國路

9748. 11812527：Insert done.	{'物件標題': '近南勢角站/靜巷/可炊/專管/大空間', '出租者': '黃先生', '出租者身份': '代理人', '聯絡電話': '0975250186', '物件型態': '公寓', '物件現況': '1房1廳1衛', '性別要求': '無特別限制'}
9749. 11850960：Insert done.	{'物件標題': '淡水捷運站旁老街精緻套房', '出租者': '黃小姐', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1547560', '物件型態': '透天厝', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9750. 11770182：Insert done.	{'物件標題': '近頂溪捷運站，中正路一樓套房出租', '出租者': '李先生', '出租者身份': '屋主', '聯絡電話': '0928-094-656', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9751. 11808722：Insert done.	{'物件標題': '交通超便利隔壁棟即頂溪捷運站', '出租者': '吳先生', '出租者身份': '屋主', '聯絡電話': '0920-462-658', '物件型態': '電梯大樓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9752. 11753725：Insert done.	{'物件標題': '裕明商圈✿有大小兩間可選✿特價中👍️', '出租者': '賴先生', '出租者身份': '代理人', '聯絡電話': '0905-539-900', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9753. 11781535：Insert done.	{'物件標題': '龍門龍濱路仁興街交义路口全新裝璜', '出租者': '鄭先生', '出租者身份': '屋主', '聯絡電話': '0922-110-306', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9754. 11806616：Insert done.	{'物件標題': '▋▋楊凱婷▋▋全新

9800. 11744134：Insert done.	{'物件標題': 'LINE會回,可貓,首月免租金', '出租者': '黃小姐', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1541058', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9801. 11804801：Insert done.	{'物件標題': '優質共居有質感的生活空間(貓咪可入住)', '出租者': '張先生', '出租者身份': '屋主', '聯絡電話': '0925-955-505', '物件型態': '公寓', '物件現況': '雅房', '性別要求': '無特別限制'}
9802. 11829954：Insert done.	{'物件標題': '永和區文化路、離捷運站近', '出租者': '陳小姐', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1546278', '物件型態': '公寓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
9803. 11846280：Insert done.	{'物件標題': '府中捷運套房(附流理台電磁爐)通風採光佳', '出租者': '林先生', '出租者身份': '代理人', '聯絡電話': '0939-330-019', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9804. 11823487：Insert done.	{'物件標題': '精美溫馨大套房', '出租者': '李先生', '出租者身份': '屋主', '聯絡電話': '0935-865-063', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9805. 11738491：Insert done.	{'物件標題': '獨立大套房,美麗溫馨,全新裝潢,廚房', '出租者': '林小姐', '出租者身份': '屋主', '聯絡電話': '0970-610-369', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
9806. 11839615：Insert done.	{'物件標題': '生活機能方便，緊

9852. 11795615：Insert done.	{'物件標題': '永安捷運站旁，緊鄰四號公園，環境優交通便', '出租者': '林小姐', '出租者身份': '屋主', '聯絡電話': '0931-075-975', '物件型態': '公寓', '物件現況': '雅房', '性別要求': '限女性'}
9853. 11705806：Insert done.	{'物件標題': '景安捷運典雅風套房出租', '出租者': '孫先生', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1538478', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
9854. 11722061：Insert done.	{'物件標題': '全新裝潢，溫馨陽光宅', '出租者': '鄭先生', '出租者身份': '代理人', '聯絡電話': '0912-886-130', '物件型態': '公寓', '物件現況': '3房2廳1衛', '性別要求': '無特別限制'}
9855. 11841441：Insert done.	{'物件標題': '頂溪站2房乾濕分離衛浴獨立洗衣機', '出租者': 'Abbie', '出租者身份': '屋主', '聯絡電話': '0955-507-898', '物件型態': '公寓', '物件現況': '2房1廳1衛', '性別要求': '無特別限制'}
9856. 11745404：Insert done.	{'物件標題': '土城捷運站3分鐘新大樓即可入住', '出租者': '鐘先生', '出租者身份': '屋主', '聯絡電話': '0923-945-489', '物件型態': '電梯大樓', '物件現況': '2房1廳1衛', '性別要求': '無特別限制'}
9857. 11841546：Insert done.	{'物件標題': '近捷運府中站電梯大樓24H警衛管理大四房', '出租者': '陳先生', '出租者身份': '代理人', '聯絡電話': '0939-926-926', '物件型態': '電梯大樓', '物件現況': '4房2廳2衛', '性別要求': '無特別限制'}
9858. 11705806：Insert done.	{'物件標題': '景安捷運

...

19071. 11753160：Insert done.	{'物件標題': '獨立陽台洗衣機漂亮美套房', '出租者': '游先生', '出租者身份': '屋主', '聯絡電話': '0928-283-291', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19072. 11768149：Insert done.	{'物件標題': '~重陽橋附近套房~', '出租者': '林先生', '出租者身份': '仲介', '聯絡電話': '0900-498-425', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19073. 11768005：Insert done.	{'物件標題': '招財景觀優質海帝四房車有傢俱', '出租者': '陳先生', '出租者身份': '仲介', '聯絡電話': '0983-986-655', '物件型態': '電梯大樓', '物件現況': '4房2廳4衛', '性別要求': '無特別限制'}
19074. 11765352：Insert done.	{'物件標題': '三房+車位、有傢俱、社區管理', '出租者': '值先生', '出租者身份': '仲介', '聯絡電話': '0978-536-458', '物件型態': '電梯大樓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
19075. 11758041：Insert done.	{'物件標題': '【海景首排景觀，舒適大兩房車】', '出租者': '廖先生', '出租者身份': '仲介', '聯絡電話': '0938-966-286', '物件型態': '電梯大樓', '物件現況': '2房2廳1衛', '性別要求': '無特別限制'}
19076. 11767502：Insert done.	{'物件標題': '安馨║景平路║近捷運║格局方正║套房', '出租者': '陳俊宏', '出租者身份': '仲介', '聯絡電話': '0987-786-758', '物件型態': '電梯大樓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
19077. 11669769：Insert done.	{'物件標題': '新板特區，大遠百旁，五鐵共

19125. 11718585：Insert done.	{'物件標題': 'A號房-刷卡管制可租金補貼裕民商圈', '出租者': '蔡先生', '出租者身份': '屋主', '聯絡電話': '0902-185-799', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19126. 11718592：Insert done.	{'物件標題': 'B號房刷卡管制可申請租金補貼裕民商圈', '出租者': '蔡先生', '出租者身份': '屋主', '聯絡電話': '0902-185-799', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19127. 11756544：Insert done.	{'物件標題': '板橋國光路地下停車位', '出租者': '詹小姐', '出租者身份': '屋主', '聯絡電話': '0960-727-352', '物件型態': '一年', '物件現況': '2坪', '性別要求': '無特別限制'}
19128. 11736879：Insert done.	{'物件標題': '板橋區近亞東醫院站亞東科大大套房', '出租者': '謝先生', '出租者身份': '仲介', '聯絡電話': '0925-128-531', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
19129. 11736885：Insert done.	{'物件標題': '板橋區近江翠國小大套房新裝潢小資首選', '出租者': '謝先生', '出租者身份': '仲介', '聯絡電話': '0925-128-531', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19130. 11741567：Insert done.	{'物件標題': '新莊區近幸福站套房獨洗、曬空間採光通風佳', '出租者': '謝先生', '出租者身份': '仲介', '聯絡電話': '0925-128-531', '物件型態': '公寓', '物件現況': '獨立套房', '性別要求': '無特別限制'}
19131. 11741568：Insert done.	{'物件標題': '新莊區近新莊棒球場家樂福

19177. 11745344：Insert done.	{'物件標題': '文化路一段第一銀行樓下坡道平面停車', '出租者': '陳先生', '出租者身份': '屋主', '聯絡電話': '0986-851-077,1541139', '物件型態': '三個月', '物件現況': '1坪', '性別要求': '無特別限制'}
19178. 11744866：Insert done.	{'物件標題': '室內地下室_平面車位(很好停泊)', '出租者': '冼先生', '出租者身份': '屋主', '聯絡電話': '0966-097-757', '物件型態': '半年', '物件現況': '5坪', '性別要求': '無特別限制'}
19179. 11744379：Insert done.	{'物件標題': '佳園路一二樓透天金店面🍎群洋太平洋佩珍', '出租者': '蔡小姐', '出租者身份': '仲介', '聯絡電話': '0971-255-998', '物件型態': '透天厝', '物件現況': '開放式格局', '性別要求': '無特別限制'}
19180. 11740553：Insert done.	{'物件標題': '全新4房+車位', '出租者': '林金泉', '出租者身份': '仲介', '聯絡電話': '0930-924-000', '物件型態': '電梯大樓', '物件現況': '4房2廳1衛', '性別要求': '無特別限制'}
19181. 11739349：Insert done.	{'物件標題': '中央路四段店面住', '出租者': '林先生', '出租者身份': '仲介', '聯絡電話': '0981-055-784', '物件型態': '透天厝', '物件現況': '開放式格局', '性別要求': '無特別限制'}
19182. 11743926：Insert done.	{'物件標題': '泰隆柏園3房+大露台~全業房屋', '出租者': '全業房屋', '出租者身份': '仲介', '聯絡電話': '', '物件型態': '電梯大樓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
19183. 11743764：Insert done.	{'物件標題': '海洋都心III優質套房⭐', '出租者':

19230. 11732632：Insert done.	{'物件標題': '邊間2房1廳低樓層', '出租者': '葉太太', '出租者身份': '屋主', '聯絡電話': '', '物件型態': '公寓', '物件現況': '2房1廳1衛', '性別要求': '無特別限制'}
19231. 11732610：Insert done.	{'物件標題': '2房1廳低樓層', '出租者': '葉太太', '出租者身份': '屋主', '聯絡電話': '', '物件型態': '公寓', '物件現況': '2房1廳1衛', '性別要求': '無特別限制'}
19232. 11731939：Insert done.	{'物件標題': '宏達民義停車場', '出租者': '客服先生', '出租者身份': '代理人', '聯絡電話': '0980156800', '物件型態': '一年', '物件現況': '4坪', '性別要求': '無特別限制'}
19233. 11731903：Insert done.	{'物件標題': '宏達五福場停車場', '出租者': '客服人員', '出租者身份': '代理人', '聯絡電話': '0980156800', '物件型態': '一年', '物件現況': '4坪', '性別要求': '無特別限制'}
19234. 11731600：Insert done.	{'物件標題': '蘆洲仁愛街車位出租7500季繳', '出租者': '李先生', '出租者身份': '屋主', '聯絡電話': '0901-234-676', '物件型態': '三個月', '物件現況': '6坪', '性別要求': '無特別限制'}
19235. 11731588：Insert done.	{'物件標題': '三重集仁街、仁勇街優惠車位出租7500', '出租者': '李先生', '出租者身份': '屋主', '聯絡電話': '0901-234-676', '物件型態': '三個月', '物件現況': '5坪', '性別要求': '無特別限制'}
19236. 11731572：Insert done.	{'物件標題': '蘆洲捷運站三民路車位出租無棚車位', '出租者': '李先生', '出租者身份': '屋主', '聯絡電話': '0901-234-676'

19283. 11709711：Insert done.	{'物件標題': '未來城三井溫馨3房美屋', '出租者': '劉小姐', '出租者身份': '仲介', '聯絡電話': '0953-700-267', '物件型態': '電梯大樓', '物件現況': '3房2廳2衛', '性別要求': '無特別限制'}
19284. 11708597：Insert done.	{'物件標題': '近迴龍捷運站美妝套房', '出租者': '陳先生', '出租者身份': '仲介', '聯絡電話': '0912-757-554', '物件型態': '公寓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19285. 11707601：Insert done.	{'物件標題': '重金打造全新裝潢大套房', '出租者': '張先生', '出租者身份': '仲介', '聯絡電話': '0981-975-716', '物件型態': '電梯大樓', '物件現況': '分租套房', '性別要求': '無特別限制'}
19286. 11672977：Insert done.	{'物件標題': '群洋太平洋佩珍🍎北大牛津博士二房+車位', '出租者': '蔡小姐', '出租者身份': '仲介', '聯絡電話': '0971-255-998', '物件型態': '電梯大樓', '物件現況': '2房2廳2衛', '性別要求': '無特別限制'}


In [67]:
#####################測試某單獨頁面專用#####################

import time
#import json

def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec;

tStart = time.time()#計時開始

rqheader = {'User-Agent': 'Custom',
            'X-CSRF-TOKEN': 'YN8YoYS0DTEAKQHAGTsbLnMt7nOuUZE5dOGZtV4W',
            'device': 'pc',
            'deviceid': '7ikinjg0thbdur20s2cueh28i3'}

itemid = '11704896'
itemurl = 'https://bff.591.com.tw/v1/house/rent/detail?id=' + str(itemid)
#print(itemurl)
response = rq.get(itemurl, headers = rqheader)
soup = BeautifulSoup(response.text)

item_data = str(soup)

item_data = item_data.replace('<html>','')
item_data = item_data.replace('<body>','')
item_data = item_data.replace('<p>','')
item_data = item_data.replace('</html>','')
item_data = item_data.replace('</body>','')
item_data = item_data.replace('</p>','')
item_data = item_data.replace('</a>','')

item_data = item_data.strip()


#以下是該死的html語法錯亂之校正
#先print到此為止的結果，並將文本與Postman解析出的正確版文本做比較，以調整錯誤的HTML標籤並符合JSON文本規範。
item_data = item_data.replace('\"\\&quot;', '\\"')
item_data = item_data.replace('\\&quot;', ';')
item_data = item_data.replace(';;\"', '\\\"')
###文本調整結束###

print(item_data)

#print(item_data)
item_data = eval(item_data)['data']
#print(item_data)

item_title = str(item_data['title'])
item_contactor = str(item_data['linkInfo']['name'])
item_crole = str(item_data['linkInfo']['roleName'])
item_contactor = str(item_contactor.replace(item_crole + ': ', ''))
item_phone = str(item_data['linkInfo']['mobile'])
#item_shape = 
item_status = str(item_data['info'][3]['value'])
item_rule = str(item_data['service']['rule'])
print(item_rule)


if ('女' in item_rule) == True :
    item_genderlim = '限女性'
elif ('男' in item_rule) == True :
    item_genderlim = '限男性'
else:
    item_genderlim = '無特別限制'



print(item_title + ' ' + item_contactor + ' ' +item_crole + ' ' + item_phone + ' ' + ' ' +item_status + ' ' + item_genderlim)
time.sleep(sleeptime(0,0,1))

#print('物件' + str(itemid) + '擷取錯誤，跳過')




{"status":0,"msg":"物件不存在","data":""}


TypeError: string indices must be integers

In [59]:
print(str(taipeicity))

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'rental_db'), 'taipeicity')
